In [ ]:
#Conexão com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import requests
import json
from openai import OpenAI
# Substitua 'your_openai_api_key' pela sua chave de API da OpenAI
client = OpenAI(api_key='*********')


In [6]:
import time

def summarize_news(news_file):
    # Carrega o conteúdo das notícias de um arquivo JSON
    with open(news_file, 'r') as file:
        news_data = json.load(file)
        news_contents = news_data['news_content']

    summaries = []

    for content in news_contents:
        # Define os dados para enviar para a API da GPT-3.5-turbo

        response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          response_format={ "type": "json_object" },
          messages=[
            {
              "role": "system",
              "content": "Summarize this news article and return in the following JSON format containing only summary."
            },
            {
              "role": "user",
              "content": f"{content}\n###"
            }
          ],
          temperature=0.3,
          max_tokens=64,
          top_p=1
        )

        summary_text = response.choices[0].message.content.strip()
        print(summary_text)
        summaries.append({
            "story": content,
            "summary": summary_text.replace('{\n    \"summary\": \"', "")
        })
        time.spleep(25)


    # Salva os resultados em um arquivo JSON
    with open('news_summaries.json', 'w') as json_file:
        json.dump({"news_summaries": summaries}, json_file)

# Chame a função com o caminho para seu arquivo de conteúdo de notícias
summarize_news('/content/drive/MyDrive/news_contents.json')

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

A porcaria da openia não funciona pq fica ramelando as requisições😠

In [7]:
!pip install -U -q google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 30.6 MB/s eta 0:00:00


In [10]:
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('API_KEY')
genai.configure(api_key=api_key)

In [34]:
import time

generation_config = {
  "temperature": 0.5, # Esse parâmetro controla a aleatoriedade do texto gerado
  # "candidate_count": 1, # Especifica o número de tentativas independentes de geração de texto (candidatos) a serem consideradas.
  "top_p": 0.90, # Controla a distribuição de probabilidade usada para amostrar a próxima palavra.
  "top_k": 10, # Limita o tamanho do vocabulário considerado para gerar a próxima palavra
  "max_output_tokens": 64 # Define o número máximo de tokens (palavras) que o texto gerado pode ter.
}

def summarize_news(news_file):
    # Carrega o conteúdo das notícias de um arquivo JSON
    with open(news_file, 'r') as file:
        news_data = json.load(file)
        news_contents = news_data['news_content']

    summaries = []
    count = 0

    for content in news_contents:
        # Define os dados para enviar para a API do genai

        # prompt = f"Summarize this news article and return in the following JSON format containing only summary.: {content}"
        layout = '{"summary": ""}'
        prompt = f'Summarize this news and return it in the following JSON format containing just a summary ({layout}). The news: {content}'
        model_RAG = genai.GenerativeModel("gemini-1.0-pro",
                                          generation_config=generation_config)
        response = model_RAG.generate_content(prompt)

        summary_text = response.text
        print(summary_text)
        summaries.append({
            "story": content,
            "summary": summary_text.replace('{\n    \"summary\": \"', "")
        })
        count = count + 1
        if count == 5:
          break
        time.sleep(15)

    # Salva os resultados em um arquivo JSON
    with open('news_summaries.json', 'w') as json_file:
        json.dump({"news_summaries": summaries}, json_file)

# Chame a função com o caminho para seu arquivo de conteúdo de notícias
summarize_news('/content/drive/MyDrive/news_contents.json')

{"summary": "An Israeli airstrike on a UN school in Gaza killed at least 18 people, including six UN staff, and injured 44 others. UNRWA, the UN agency for Palestinian humanitarian relief, confirmed the deaths of its employees and condemned the attack. The Israel Defense Forces (IDF)
{"summary": "Michel Barnier, the EU's former Brexit negotiator, has been appointed as France's new Prime Minister. Barnier, a staunch Europhile and member of the Republicans party, will face the challenge of forming a stable government after inconclusive parliamentary elections. The far-right National Rally has expressed willingness
{"summary": "The provided text does not contain any news content, so I cannot provide a summary."}
{"summary": "Russia has launched a counteroffensive in the Kursk border region, where Ukrainian forces previously conducted a surprise cross-border incursion. Ukrainian President Zelensky acknowledges the Russian actions as part of the 'Ukrainian plan' and confirms that Ukraine ha